In [1]:
import torch 
import torch.nn as nn
import os
from torchvision.models import alexnet
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from typing import Tuple, List
from PIL import Image
import json

In [2]:
PATH_TO_CHECKPOINT = "/home/kenzo/experiments/alexnet-pretrained-super_aug-200ep/fold9/MyAlexNetPretrained_200.pth"
PATH_TO_IMAGES = "/home/kenzo/datasets/embrapa/Fake-P2-dataset"
PATH_TO_LABELS = "/hd4t/home/biomassa/exp/GSD05/2019-01-23/labels/annotation.json"

In [3]:
device = "cuda:2"

# Carregando modelo pre-treinado

In [4]:
# TUDO ISSO PARA CARREGAR UMA ALEXNET DISGRAÇAAA

class BaseConvNet(nn.Module):
    def __init__(self, name: str = "Model",
                       features: nn.Sequential = None, 
                       classifier: nn.Sequential = None):
        super().__init__()
        self.name = name
        self._features = features
        self._classifier = classifier
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self._features(x)
        print(x.shape)
        x = self._classifier(x)
        
        return x


class AlexNet(BaseConvNet):
    def __init__(self, nClasses):
        features = nn.Sequential(
            # first layer
            nn.Conv2d(3, 96, 11, stride=4),
            nn.ReLU(inplace=False),
            nn.MaxPool2d(3, stride=2),
            # second layer
            nn.Conv2d(96, 256, 5, stride=1, padding=2),
            nn.ReLU(inplace=False),
            nn.MaxPool2d(3, stride=2),
            # third layer
            nn.Conv2d(256, 384, 3, padding=1),
            nn.ReLU(inplace=False),
            # fourth layer
            nn.Conv2d(384, 384, 3, padding=1),
            nn.ReLU(inplace=False),
            # fifth layer
            nn.Conv2d(384, 256, 3, padding=1),
            nn.ReLU(inplace=False),
            nn.MaxPool2d(3, stride=2))
        
        classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(6*6*256, 4096, bias=True),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 4096, bias=True),
            nn.ReLU(inplace=True),
            nn.Linear(4096, nClasses, bias=True)
        )

        super().__init__("AlexNet", features, classifier)

In [5]:
def get_imagent_alexNet():
    net = alexnet(pretrained=True)
    
    regressor = nn.Sequential(
        nn.Flatten(),
        nn.Linear(10*6*256, 4096, bias=True),
        nn.ReLU(inplace=True),
        nn.Linear(4096, 4096, bias=True),
        nn.ReLU(inplace=True),
        nn.Linear(4096, 1, bias=True))
    net.classifier = regressor

    net.name = "ImagenetAlexNet"
    return net

def get_alexNet():
    net = AlexNet(1)
    
    regressor = nn.Sequential(
            nn.Flatten(),
            nn.Linear(10*6*256, 4096, bias=True),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 4096, bias=True),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 1, bias=True))
    
    net._classifier = regressor

    return net

def get_myalexnet_pretrained():
    net = get_alexNet()
    imagenet_alexnet = get_imagent_alexNet()
    net._features = imagenet_alexnet.features
    net.name = "MyAlexNetPretrained"

    return net

In [6]:
model = get_myalexnet_pretrained() # mas que monstruosidade é essa

In [7]:
model.load_state_dict(torch.load(PATH_TO_CHECKPOINT, map_location=device)["state_dict"], strict=False)

<All keys matched successfully>

# Preparando um dataset para inferência

In [8]:
def get_transforms(xstats: dict):

    normalize = transforms.Normalize(mean=xstats["mean"], std=xstats["std"])

    t = transforms.Compose([
            transforms.Resize(227),
            transforms.ToTensor(),
            normalize])
            
    return t

In [9]:
class InferenceDataset(Dataset):

    def __init__(self,
                 source_dir: str,
                 transforms: transforms.Compose):

        self.images = [os.path.join(source_dir, image) for image in os.listdir(source_dir)]
        self.transform = transforms

    def __len__(self) -> int:

        return len(self.images)

    def __getitem__(self, i) -> torch.Tensor:

        image_path = self.images[i]
        image = Image.open(image_path)

        return self.transform(image)

        

In [10]:
annotations = json.load(open(PATH_TO_LABELS, "r"))
xstats = annotations["statistics"]["x"]

In [11]:
transforms = get_transforms(xstats)

In [12]:
ds = InferenceDataset(PATH_TO_IMAGES, transforms)
dl = DataLoader(ds, batch_size=32, shuffle=False)

# Aplicando predição em imagens novas

In [13]:
def predict(model, inference_dl, device = "cpu"):
    predictions = []
    with torch.no_grad():
        for image_batch in inference_dl:

            image_batch = image_batch.to(device)
            preds = model(image_batch).view(-1)
            predictions.extend(preds.tolist())

    return predictions


In [14]:
preds = predict(model.to(device), dl, device)

torch.Size([32, 256, 10, 6])
torch.Size([1, 256, 10, 6])


In [15]:
import pandas as pd

In [16]:
preds_df = pd.DataFrame()
# preds_df["images"] = ds.images
preds_df["Predicao"] = preds
preds_df["Parcela"] = [int(name[59:62]) for name in ds.images]

In [17]:
preds_df

,Predicao,Parcela
0,6209.011719,223
1,5034.480957,164
2,6704.708008,203
3,10354.244141,21
4,11809.332031,166
5,9114.894531,329
6,7003.057129,69
7,5908.583984,304
8,14603.240234,277
9,10123.066406,60
